<a href="https://colab.research.google.com/github/Tanvir-tareq/python/blob/main/water_jug_problem_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Water Jug Problem Solver with Interactive Visualization in Google Colab
# This creates a complete web app that runs in your Colab notebook!
# Now showing ALL steps in two different visualizations

# Install required packages
!pip install -q ipywidgets matplotlib networkx
!pip install -q gradio
!jupyter nbextension enable --py widgetsnbextension

import gradio as gr
import matplotlib.pyplot as plt
import networkx as nx
from collections import deque
import time
import numpy as np
from matplotlib.patches import Rectangle, Circle, Polygon
import matplotlib.animation as animation
from IPython.display import HTML, display, Image
import warnings
import base64
import io
import os
warnings.filterwarnings('ignore')

class WaterJugSolver:
    """Solves water jug problem using BFS algorithm"""

    def __init__(self):
        self.solution_path = []
        self.states_visited = 0
        self.solve_time = 0

    def bfs(self, x_capacity, y_capacity, target):
        """Breadth-First Search algorithm"""
        start_time = time.time()

        start = (0, 0)
        queue = deque()
        queue.append((start, [start]))
        visited = set()
        visited.add(start)

        while queue:
            (x, y), path = queue.popleft()
            self.states_visited += 1

            # Check if we found the solution
            if x == target or y == target:
                self.solution_path = path
                self.solve_time = time.time() - start_time
                return True

            # Generate all possible next states
            next_states = []

            # Rule 1: Fill jug 1
            if x < x_capacity:
                next_states.append((x_capacity, y))

            # Rule 2: Fill jug 2
            if y < y_capacity:
                next_states.append((x, y_capacity))

            # Rule 3: Empty jug 1
            if x > 0:
                next_states.append((0, y))

            # Rule 4: Empty jug 2
            if y > 0:
                next_states.append((x, 0))

            # Rule 5: Pour from jug 1 to jug 2
            if x > 0 and y < y_capacity:
                pour = min(x, y_capacity - y)
                next_states.append((x - pour, y + pour))

            # Rule 6: Pour from jug 2 to jug 1
            if y > 0 and x < x_capacity:
                pour = min(y, x_capacity - x)
                next_states.append((x + pour, y - pour))

            # Add unvisited states to queue
            for state in next_states:
                if state not in visited:
                    visited.add(state)
                    queue.append((state, path + [state]))

        self.solve_time = time.time() - start_time
        return False

    def get_action_description(self, prev_state, current_state, x_cap, y_cap):
        """Get description of action between states"""
        prev_x, prev_y = prev_state
        curr_x, curr_y = current_state

        if curr_x == x_cap and prev_x != x_cap and curr_y == prev_y:
            return f"Fill Jug 1 ({x_cap}L)"
        elif curr_y == y_cap and prev_y != y_cap and curr_x == prev_x:
            return f"Fill Jug 2 ({y_cap}L)"
        elif curr_x == 0 and prev_x != 0 and curr_y == prev_y:
            return "Empty Jug 1"
        elif curr_y == 0 and prev_y != 0 and curr_x == prev_x:
            return "Empty Jug 2"
        elif curr_x < prev_x and curr_y > prev_y:
            return f"Pour {prev_x - curr_x}L from Jug 1 to Jug 2"
        elif curr_x > prev_x and curr_y < prev_y:
            return f"Pour {prev_y - curr_y}L from Jug 2 to Jug 1"
        else:
            return "Initial State"

    def get_solution_details(self, x_capacity, y_capacity, target):
        """Get detailed solution with actions"""
        if not self.solution_path:
            return []

        details = []
        for i, (x, y) in enumerate(self.solution_path):
            if i == 0:
                action = "Start with empty jugs"
            else:
                action = self.get_action_description(
                    self.solution_path[i-1],
                    (x, y),
                    x_capacity,
                    y_capacity
                )

            target_reached = (x == target or y == target)
            details.append({
                'step': i,
                'jug1': x,
                'jug2': y,
                'action': action,
                'target_reached': target_reached
            })

        return details

# ============================================================================
# OPTION 1: ALL STEPS GALLERY
# ============================================================================

def create_all_steps_gallery(solution_details, x_capacity, y_capacity):
    """Create a gallery of ALL steps in the solution"""
    if not solution_details:
        return None

    n_steps = len(solution_details)
    n_cols = min(4, n_steps)  # Maximum 4 columns for better readability
    n_rows = (n_steps + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows))
    fig.suptitle(f'Complete Solution: All {n_steps} Steps',
                 fontsize=16, fontweight='bold', y=1.02)

    # Handle single plot case
    if n_rows == 1 and n_cols == 1:
        axes = np.array([[axes]])
    elif n_rows == 1:
        axes = axes.reshape(1, -1)
    elif n_cols == 1:
        axes = axes.reshape(-1, 1)

    # Colors
    water_color = '#4a9fff'
    jug_color = '#2e4a7a'
    highlight_color = '#ff6b6b'

    max_height = max(x_capacity, y_capacity)

    for idx, detail in enumerate(solution_details):
        row = idx // n_cols
        col = idx % n_cols

        ax = axes[row, col]
        x_amount = detail['jug1']
        y_amount = detail['jug2']

        # Clear the axis
        ax.clear()
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(0, max_height + 1)

        # Draw two jugs side by side
        jug_width = 0.4
        jug_spacing = 1.0  # Space between jugs

        # Jug 1 (left)
        jug1_x = -jug_spacing/2
        jug1_height = x_capacity
        water1_height = (x_amount / x_capacity) * jug1_height if x_capacity > 0 else 0

        # Jug outline
        ax.add_patch(Rectangle(
            (jug1_x - jug_width/2, 0), jug_width, jug1_height,
            fill=False, edgecolor=jug_color, linewidth=2
        ))

        # Water
        ax.add_patch(Rectangle(
            (jug1_x - jug_width/2, 0), jug_width, water1_height,
            color=water_color, alpha=0.8, edgecolor='darkblue', linewidth=1
        ))

        # Water level line
        ax.hlines(y=water1_height,
                 xmin=jug1_x - jug_width/2,
                 xmax=jug1_x + jug_width/2,
                 colors='red', linewidth=1.5)

        # Label
        ax.text(jug1_x, jug1_height + 0.3,
               f'J1: {x_amount}/{x_capacity}L',
               ha='center', fontsize=9, fontweight='bold')

        # Jug 2 (right)
        jug2_x = jug_spacing/2
        jug2_height = y_capacity
        water2_height = (y_amount / y_capacity) * jug2_height if y_capacity > 0 else 0

        # Jug outline
        ax.add_patch(Rectangle(
            (jug2_x - jug_width/2, 0), jug_width, jug2_height,
            fill=False, edgecolor=jug_color, linewidth=2
        ))

        # Water
        ax.add_patch(Rectangle(
            (jug2_x - jug_width/2, 0), jug_width, water2_height,
            color=water_color, alpha=0.8, edgecolor='darkblue', linewidth=1
        ))

        # Water level line
        ax.hlines(y=water2_height,
                 xmin=jug2_x - jug_width/2,
                 xmax=jug2_x + jug_width/2,
                 colors='red', linewidth=1.5)

        # Label
        ax.text(jug2_x, jug2_height + 0.3,
               f'J2: {y_amount}/{y_capacity}L',
               ha='center', fontsize=9, fontweight='bold')

        # Title for the subplot
        title = f"Step {detail['step']}"
        if detail['target_reached']:
            title += " 🎯"
            # Highlight the plot
            for spine in ax.spines.values():
                spine.set_edgecolor('green')
                spine.set_linewidth(3)

        ax.set_title(title, fontsize=11, fontweight='bold', pad=10)

        # Add action description
        action = detail['action']
        if len(action) > 30:  # Truncate long actions
            action = action[:27] + "..."

        # Place text below the jugs
        ax.text(0, -0.5, action,
               ha='center', fontsize=8, wrap=True,
               bbox=dict(boxstyle="round,pad=0.3",
                        facecolor='lightyellow',
                        alpha=0.7))

        # Hide axis
        ax.set_xticks([])
        ax.set_yticks([])
        for spine in ['top', 'right', 'bottom', 'left']:
            ax.spines[spine].set_visible(False)

    # Turn off any unused axes
    for idx in range(n_steps, n_rows * n_cols):
        row = idx // n_cols
        col = idx % n_cols
        axes[row, col].axis('off')

    plt.tight_layout()
    return fig

# ============================================================================
# OPTION 2: CORRECTED ANIMATED VISUALIZATION
# ============================================================================

def create_animation_gif(solution_details, x_capacity, y_capacity):
    """Create an animated GIF showing all steps with BOTH jugs visible"""
    if not solution_details or len(solution_details) == 0:
        return None

    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 8))

    # Colors
    water_color = '#4a9fff'
    jug_color = '#2e4a7a'
    bg_color = '#f8f9fa'

    # Set background
    fig.patch.set_facecolor(bg_color)
    ax1.set_facecolor(bg_color)
    ax2.set_facecolor(bg_color)

    # Store artists for updating
    water_rects = []
    level_lines = []
    text_labels = []
    jug_outlines = []

    # Initialize jug positions and sizes
    jug_width = 0.6
    jug_spacing = 1.2  # Space between jug centers
    jug1_center = -jug_spacing/2  # Left jug center
    jug2_center = jug_spacing/2   # Right jug center

    jug1_height = x_capacity
    jug2_height = y_capacity
    max_height = max(jug1_height, jug2_height)

    # Function to initialize plot
    def init():
        nonlocal water_rects, level_lines, text_labels, jug_outlines

        # Clear axes
        ax1.clear()
        ax2.clear()

        # Setup axes limits
        ax1.set_xlim(-jug_spacing, jug_spacing)
        ax1.set_ylim(0, max_height + 1.5)
        ax1.set_aspect('auto')

        # Draw jug outlines (these won't change)
        jug1_outline = Rectangle(
            (jug1_center - jug_width/2, 0), jug_width, jug1_height,
            fill=False, edgecolor=jug_color, linewidth=3, zorder=2
        )
        jug2_outline = Rectangle(
            (jug2_center - jug_width/2, 0), jug_width, jug2_height,
            fill=False, edgecolor=jug_color, linewidth=3, zorder=2
        )

        ax1.add_patch(jug1_outline)
        ax1.add_patch(jug2_outline)
        jug_outlines.extend([jug1_outline, jug2_outline])

        # Add initial water (empty)
        water1 = Rectangle(
            (jug1_center - jug_width/2, 0), jug_width, 0,
            color=water_color, alpha=0.8, zorder=1
        )
        water2 = Rectangle(
            (jug2_center - jug_width/2, 0), jug_width, 0,
            color=water_color, alpha=0.8, zorder=1
        )

        ax1.add_patch(water1)
        ax1.add_patch(water2)
        water_rects.extend([water1, water2])

        # Add initial level lines (at bottom)
        line1, = ax1.plot(
            [jug1_center - jug_width/2, jug1_center + jug_width/2],
            [0, 0], 'r-', linewidth=2, zorder=3
        )
        line2, = ax1.plot(
            [jug2_center - jug_width/2, jug2_center + jug_width/2],
            [0, 0], 'r-', linewidth=2, zorder=3
        )
        level_lines.extend([line1, line2])

        # Add text labels
        text1 = ax1.text(
            jug1_center, jug1_height + 0.8,
            f'Jug 1: 0/{x_capacity}L',
            ha='center', fontsize=12, fontweight='bold', zorder=4
        )
        text2 = ax1.text(
            jug2_center, jug2_height + 0.8,
            f'Jug 2: 0/{y_capacity}L',
            ha='center', fontsize=12, fontweight='bold', zorder=4
        )
        text_labels.extend([text1, text2])

        # Add capacity labels
        ax1.text(jug1_center, -0.3, f'Capacity: {x_capacity}L',
                ha='center', fontsize=10, fontstyle='italic')
        ax1.text(jug2_center, -0.3, f'Capacity: {y_capacity}L',
                ha='center', fontsize=10, fontstyle='italic')

        # Setup step table in ax2
        ax2.axis('off')
        ax2.set_xlim(0, 10)
        ax2.set_ylim(0, len(solution_details) + 2)

        # Add table header
        ax2.text(1, len(solution_details) + 1, 'Step',
                fontsize=12, fontweight='bold', ha='center')
        ax2.text(3, len(solution_details) + 1, 'Jug 1',
                fontsize=12, fontweight='bold', ha='center')
        ax2.text(5, len(solution_details) + 1, 'Jug 2',
                fontsize=12, fontweight='bold', ha='center')
        ax2.text(7, len(solution_details) + 1, 'Action',
                fontsize=12, fontweight='bold', ha='center')

        ax1.set_title('Water Jugs State', fontsize=14, fontweight='bold', pad=20)
        ax2.set_title('Step-by-Step Table', fontsize=14, fontweight='bold', pad=20)

        ax1.grid(True, alpha=0.3, linestyle='--')

        return water_rects + level_lines + text_labels + jug_outlines

    # Function to update each frame
    def update(frame):
        nonlocal water_rects, level_lines, text_labels

        detail = solution_details[frame]
        x_amount = detail['jug1']
        y_amount = detail['jug2']

        # Update water levels
        water1_height = (x_amount / x_capacity) * jug1_height
        water2_height = (y_amount / y_capacity) * jug2_height

        water_rects[0].set_height(water1_height)
        water_rects[1].set_height(water2_height)

        # Update level lines
        level_lines[0].set_ydata([water1_height, water1_height])
        level_lines[1].set_ydata([water2_height, water2_height])

        # Update text labels
        text_labels[0].set_text(f'Jug 1: {x_amount}/{x_capacity}L')
        text_labels[1].set_text(f'Jug 2: {y_amount}/{y_capacity}L')

        # Update main title
        fig.suptitle(f'Step {frame}: {detail["action"]}',
                    fontsize=16, fontweight='bold', y=0.98)

        # Highlight target reached
        if detail['target_reached']:
            ax1.text(0, max_height + 1.2, '🎯 TARGET REACHED! 🎯',
                    ha='center', fontsize=14, color='green',
                    fontweight='bold', zorder=5,
                    bbox=dict(boxstyle="round,pad=0.5",
                             facecolor='white',
                             edgecolor='green',
                             alpha=0.8))

        # Clear and redraw step table in ax2
        ax2.clear()
        ax2.axis('off')
        ax2.set_xlim(0, 10)
        ax2.set_ylim(0, len(solution_details) + 2)

        # Draw table headers
        ax2.text(1, len(solution_details) + 1, 'Step',
                fontsize=12, fontweight='bold', ha='center')
        ax2.text(3, len(solution_details) + 1, 'Jug 1',
                fontsize=12, fontweight='bold', ha='center')
        ax2.text(5, len(solution_details) + 1, 'Jug 2',
                fontsize=12, fontweight='bold', ha='center')
        ax2.text(7, len(solution_details) + 1, 'Action',
                fontsize=12, fontweight='bold', ha='center')

        # Draw all steps, highlighting current step
        for i, step_detail in enumerate(solution_details):
            y_pos = len(solution_details) - i

            # Highlight current step
            if i == frame:
                # Draw background highlight
                ax2.add_patch(Rectangle(
                    (0.5, y_pos - 0.3), 9, 0.8,
                    facecolor='lightyellow', alpha=0.5,
                    edgecolor='orange', linewidth=2, zorder=1
                ))
                color = 'red'
                weight = 'bold'
            else:
                color = 'black'
                weight = 'normal'

            # Step number
            ax2.text(1, y_pos, f"{step_detail['step']}",
                    fontsize=10, color=color, fontweight=weight, ha='center', zorder=2)

            # Jug 1 amount
            jug1_color = 'blue' if i == frame else color
            ax2.text(3, y_pos, f"{step_detail['jug1']}L",
                    fontsize=10, color=jug1_color, fontweight=weight, ha='center', zorder=2)

            # Jug 2 amount
            jug2_color = 'blue' if i == frame else color
            ax2.text(5, y_pos, f"{step_detail['jug2']}L",
                    fontsize=10, color=jug2_color, fontweight=weight, ha='center', zorder=2)

            # Action (truncated if too long)
            action = step_detail['action']
            if len(action) > 25:
                action = action[:22] + "..."
            ax2.text(7, y_pos, action,
                    fontsize=9, color=color, fontweight=weight, ha='center', zorder=2)

            # Target indicator
            if step_detail['target_reached']:
                ax2.text(8.5, y_pos, "🎯", fontsize=12, ha='center', zorder=2)

        return water_rects + level_lines + text_labels + jug_outlines

    # Create animation
    anim = animation.FuncAnimation(
        fig, update, frames=len(solution_details),
        init_func=init, blit=False, repeat=True,
        interval=1500  # 1.5 seconds per frame
    )

    # Save animation as GIF
    gif_path = f"water_jug_animation_{int(time.time())}.gif"

    # Use Pillow writer for GIF
    try:
        from matplotlib.animation import PillowWriter

        writer = PillowWriter(fps=1, bitrate=1800)
        anim.save(gif_path, writer=writer, dpi=100, savefig_kwargs={'facecolor': bg_color})

        print(f"✅ Animation saved as: {gif_path}")

    except Exception as e:
        print(f"⚠️ Error saving GIF: {e}")
        # Fallback: save as HTML
        html_path = f"water_jug_animation_{int(time.time())}.html"
        html_content = anim.to_jshtml()
        with open(html_path, 'w') as f:
            f.write(html_content)
        print(f"✅ Animation saved as HTML: {html_path}")
        gif_path = html_path

    plt.close(fig)

    return gif_path

# ============================================================================
# OPTION 3: STATE SPACE GRAPH
# ============================================================================

def create_state_space_graph(solution_path, x_capacity, y_capacity):
    """Create state space graph visualization"""
    if not solution_path:
        return None

    fig, ax = plt.subplots(figsize=(10, 8))

    G = nx.DiGraph()

    # Add nodes and edges
    for i in range(len(solution_path) - 1):
        from_state = solution_path[i]
        to_state = solution_path[i + 1]

        from_label = f"({from_state[0]},{from_state[1]})"
        to_label = f"({to_state[0]},{to_state[1]})"

        G.add_node(from_label)
        G.add_node(to_label)

        # Determine edge label
        if to_state[0] == x_capacity and from_state[0] != x_capacity:
            label = f"Fill J1"
        elif to_state[1] == y_capacity and from_state[1] != y_capacity:
            label = f"Fill J2"
        elif to_state[0] == 0 and from_state[0] != 0:
            label = f"Empty J1"
        elif to_state[1] == 0 and from_state[1] != y_capacity:
            label = f"Empty J2"
        elif to_state[0] < from_state[0]:
            label = f"Pour\n{from_state[0]-to_state[0]}L\nJ1→J2"
        else:
            label = f"Pour\n{from_state[1]-to_state[1]}L\nJ2→J1"

        G.add_edge(from_label, to_label, label=label)

    # Create layout
    pos = nx.spring_layout(G, seed=42, k=2, iterations=50)

    # Draw nodes
    node_colors = []
    node_sizes = []

    for node in G.nodes():
        # Extract coordinates from label
        coords = node.strip('()').split(',')
        x, y = int(coords[0]), int(coords[1])

        # Determine node color and size
        if x == 0 and y == 0:
            node_colors.append('lightgreen')  # Start
            node_sizes.append(2000)
        elif node == f"({solution_path[-1][0]},{solution_path[-1][1]})":
            node_colors.append('#ff9999')  # Goal (reddish)
            node_sizes.append(2000)
        elif node in [f"({s[0]},{s[1]})" for s in solution_path]:
            node_colors.append('lightblue')  # Solution path
            node_sizes.append(1500)
        else:
            node_colors.append('lightgray')
            node_sizes.append(1000)

    nx.draw_networkx_nodes(G, pos, node_size=node_sizes,
                          node_color=node_colors,
                          edgecolors='black', linewidths=2, ax=ax)

    # Draw edges
    edge_colors = []
    edge_widths = []

    for u, v in G.edges():
        if (u in [f"({s[0]},{s[1]})" for s in solution_path] and
            v in [f"({s[0]},{s[1]})" for s in solution_path]):
            edge_colors.append('blue')
            edge_widths.append(3)
        else:
            edge_colors.append('gray')
            edge_widths.append(1)

    nx.draw_networkx_edges(G, pos, edge_color=edge_colors,
                          width=edge_widths, arrows=True,
                          arrowsize=20, connectionstyle="arc3,rad=0.1",
                          ax=ax)

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=10,
                           font_weight='bold', ax=ax)

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels,
                                font_size=8, font_color='darkred',
                                ax=ax)

    # Add legend
    ax.plot([], [], 'o', markersize=10, markerfacecolor='lightgreen',
            label='Start (0,0)', markeredgecolor='black')
    ax.plot([], [], 'o', markersize=10, markerfacecolor='#ff9999',
            label='Goal/Target', markeredgecolor='black')
    ax.plot([], [], 'o', markersize=10, markerfacecolor='lightblue',
            label='Solution Path', markeredgecolor='black')
    ax.plot([], [], 'o', markersize=10, markerfacecolor='lightgray',
            label='Visited States', markeredgecolor='black')

    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

    ax.set_title("State Space Graph (BFS Path)",
                 fontsize=16, fontweight='bold')
    ax.axis('off')
    plt.tight_layout()

    return fig

# ============================================================================
# MAIN SOLVER FUNCTION (UPDATED)
# ============================================================================

def solve_and_visualize_all(x_capacity, y_capacity, target):
    """Main function to solve and visualize water jug problem with ALL steps"""

    # Validate inputs
    if x_capacity <= 0 or y_capacity <= 0 or target <= 0:
        error_msg = "❌ Error: All values must be positive integers!"
        return error_msg, None, None, None

    if target > max(x_capacity, y_capacity):
        error_msg = f"❌ Error: Target ({target}L) cannot be larger than jug capacities!"
        return error_msg, None, None, None

    # Solve the problem
    solver = WaterJugSolver()
    solution_exists = solver.bfs(x_capacity, y_capacity, target)

    if not solution_exists:
        no_solution_msg = "❌ No solution exists for these parameters!"
        no_solution_msg += "\n\n💡 Try different jug capacities or target volume."
        return no_solution_msg, None, None, None

    # Get solution details
    solution_details = solver.get_solution_details(x_capacity, y_capacity, target)

    # Create detailed output text
    output_text = f"""🎯 **SOLUTION FOUND!** 🎯

**Problem Configuration:**
- Jug 1 Capacity: {x_capacity}L
- Jug 2 Capacity: {y_capacity}L
- Target Volume: {target}L

**Algorithm Statistics:**
- Total Steps in Solution: {len(solution_details) - 1}
- States Visited During Search: {solver.states_visited}
- Solving Time: {solver.solve_time:.4f} seconds
- Optimal Path Length: {len(solver.solution_path)} states

**Optimal Path (State Sequence):**
{[f'({x},{y})' for (x, y) in solver.solution_path]}

**Detailed Step-by-Step Solution:**
"""

    for detail in solution_details:
        step_marker = f"Step {detail['step']:02d}:"
        state_info = f"Jug1 = {detail['jug1']:2}L, Jug2 = {detail['jug2']:2}L"
        action_info = f"→ {detail['action']}"

        step_line = f"{step_marker} {state_info} {action_info}"

        if detail['target_reached']:
            step_line += " 🎯 TARGET REACHED!"

        output_text += step_line + "\n"

    output_text += f"\n**Total Operations Required: {len(solution_details) - 1}**"

    # Create Option 1: All Steps Gallery
    fig_gallery = create_all_steps_gallery(solution_details, x_capacity, y_capacity)

    # Create Option 2: Animated GIF (FIXED - Now shows both jugs properly)
    gif_path = create_animation_gif(solution_details, x_capacity, y_capacity)

    # Create Option 3: State Space Graph
    fig_graph = create_state_space_graph(solver.solution_path, x_capacity, y_capacity)

    # Return all outputs
    return output_text, fig_gallery, gif_path, fig_graph

# ============================================================================
# GRADIO INTERFACE (UPDATED)
# ============================================================================

# Custom CSS for better appearance
custom_css = """
.gradio-container {
    max-width: 1400px !important;
}
h1 {
    text-align: center;
    color: #2c3e50;
}
.output-image img {
    max-width: 100%;
    border-radius: 10px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
}
.output-text textarea {
    font-family: 'Courier New', monospace;
    font-size: 12px;
}
"""

# Create the Gradio interface
demo = gr.Interface(
    fn=solve_and_visualize_all,
    inputs=[
        gr.Number(
            label="🔴 Jug 1 Capacity (Liters)",
            value=4,
            minimum=1,
            maximum=50,
            info="Enter the capacity of the first jug"
        ),
        gr.Number(
            label="🔵 Jug 2 Capacity (Liters)",
            value=3,
            minimum=1,
            maximum=50,
            info="Enter the capacity of the second jug"
        ),
        gr.Number(
            label="🎯 Target Volume (Liters)",
            value=2,
            minimum=1,
            maximum=50,
            info="Enter the desired amount of water to measure"
        )
    ],
    outputs=[
        gr.Textbox(
            label="📋 Solution Details",
            lines=25,
            elem_classes="output-text"
        ),
        gr.Plot(
            label="🖼️ All Steps Gallery",
            elem_classes="output-image"
        ),
        gr.Image(
            label="🎬 Animated Solution",
            type="filepath",
            elem_classes="output-image"
        ),
        gr.Plot(
            label="🗺️ State Space Graph",
            elem_classes="output-image"
        )
    ],
    title="🎯 Water Jug Problem Visualizer - Complete Solution 🎯",
    description="""**Solve the classic water jug puzzle using BFS algorithm!**

This interactive solver shows you:
1. 📋 **Detailed solution steps** with actions
2. 🖼️ **All steps gallery** - every step visualized at once
3. 🎬 **Animated solution** - watch the solution unfold step by step
4. 🗺️ **State space graph** - see the search space and optimal path

**How to use:**
1. Enter jug capacities and target volume
2. Click Submit
3. Explore all visualization tabs
4. Try different examples or create your own puzzle!

**Classic Puzzles:**""",
    theme=gr.themes.Soft(),
    css=custom_css,
    examples=[
        [4, 3, 2],  # Classic example (4L & 3L -> 2L)
        [5, 3, 4],  # 5L & 3L -> 4L
        [7, 5, 6],  # 7L & 5L -> 6L
        [8, 6, 4],  # 8L & 6L -> 4L
        [9, 4, 6],  # 9L & 4L -> 6L
        [10, 7, 3], # 10L & 7L -> 3L
    ],
    allow_flagging="never"
)

# Launch the app
print("=" * 70)
print("🚀 LAUNCHING WATER JUG PROBLEM VISUALIZER")
print("=" * 70)
print("\n📊 Features Included:")
print("   1. ✅ All Steps Gallery - See every step at once")
print("   2. ✅ Animated Solution - Watch step-by-step animation (FIXED!)")
print("   3. ✅ State Space Graph - Visualize the search process")
print("   4. ✅ Detailed Solution - Step-by-step instructions")
print("\n🎬 ANIMATION FIX: Both jugs are now properly visible!")
print("\n💡 Try the examples or create your own puzzle!")
print("⏳ The interface will open below...")
print("=" * 70)

# Launch with sharing enabled (creates public link)
try:
    demo.launch(share=True, debug=True)
except Exception as e:
    print(f"\n⚠️ Could not create public link. Launching locally instead...")
    print(f"Error: {e}")
    demo.launch(share=False, debug=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
🚀 LAUNCHING WATER JUG PROBLEM VISUALIZER

📊 Features Included:
   1. ✅ All Steps Gallery - See every step at once
   2. ✅ Animated Solution - Watch step-by-step animation (FIXED!)
   3. ✅ State Space Graph - Visualize the search process
   4. ✅ Detailed Solution - Step-by-step instructions

🎬 ANIMATION FIX: Both jugs are now properly visible!

💡 Try the examples or create your own puzzle!
⏳ The interface will open below...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ad21e57cb619ee6a47.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Animation saved as: water_jug_animation_1770547705.gif


# New Section